# Word detection

## Libraries required

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
import os 
os.chdir('drive/My Drive/CS3244 PROJECT/CS3244-ML')

In [40]:
!pwd # which directory u are at

/content


In [52]:
ls # items in the directory

 cropped2x.png    OpenCV2Models/      README.md        upscaled8x.png
 ImageOuput/      Our-Project.ipynb   recognized.txt  'Word Detection.ipynb'
 input.jpg        ProcessedData.csv   SamplePosters/
 MovieGenre.csv   pytesseract/        upscaled2x.png


In [9]:
# !pip install Pillow
!pip install pytesseract
# !pip install opencv-python
! apt install tesseract-ocr
! apt install libtesseract-dev
!sudo apt install tesseract-ocr
!sudo apt install pytesseract
!pip install --upgrade opencv-python
!pip install --upgrade opencv-contrib-python
# if u get any error for this block of code, just rerun it again.

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=09c47e75252a0010bc6336fb8a02774f2d44337cdee236c626b52d545b9eb79e
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
Readi

In [11]:
try:
    from PIL import Image
except ImportError:
    import Image

import pytesseract
from pytesseract import Output
import urllib.request    
import pandas as pd
import numpy as np
import cv2
from cv2 import dnn_superres

In [12]:
pytesseract.get_tesseract_version()

LooseVersion ('4.0.0-beta.1')

In [ ]:
# from os import listdir
# from os.path import isfile, join
# onlyfiles = [f for f in listdir('./') if isfile(join('./', f))]
# print(onlyfiles)

In [13]:
# get grayscale image
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# noise removal
def remove_noise(image):
    return cv2.medianBlur(image,5)
 
#thresholding
def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

#dilation
def dilate(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.dilate(image, kernel, iterations = 1)
    
#erosion
def erode(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.erode(image, kernel, iterations = 1)

#opening - erosion followed by dilation
def opening(image):
    kernel = np.ones((5,5),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

#canny edge detection
def canny(image):
    return cv2.Canny(image, 100, 200)

#skew correction
def deskew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated

#template matching
def match_template(image, template):
    return cv2.matchTemplate(image, template, cv2.TM_CCOEFF_NORMED) 

In [53]:
#Intialise the model to increase the size of the model
path = "OpenCV2Models/FSRCNN_x3.pb" 
sr = dnn_superres.DnnSuperResImpl_create()
sr.readModel(path)

# Set the desired model and scale to get correct pre- and post-processing
sr.setModel("fsrcnn", 3);






In [54]:
def url_to_image(url):
    # download the image, convert it to a NumPy array, and then read
    # it into OpenCV format
    try:
        with urllib.request.urlopen(url) as url:
            resp = url.read()
            image = np.asarray(bytearray(resp), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_COLOR)
            image = sr.upsample(image) # triple the size of the image before returning it 
        # return the image
        # print(type(image))
            return image
    except:
        return None

movie_poster_data = pd.read_csv('./MovieGenre.csv', encoding = "ISO-8859-1")
movie_poster_url = movie_poster_data['Poster']
movie_poster_url_list = movie_poster_url.tolist()

In [55]:
image_list = [url_to_image(url) for url in movie_poster_url_list[:100]]


# count = 0
# for url in movie_poster_url_list:
#     if count == 100:
#         break
#     count += 1
# #     print(count)
#     image_list.append(url_to_image(url))

In [ ]:
movie_poster_words = movie_poster_data.filter(items=['imdbId','Imdb Link','Title','Poster'])

# movie_poster_words['imdbId'] = movie_poster_data['imdbId']
# movie_poster_words['Imdb Link'] = movie_poster_data['Imdb Link']
# movie_poster_words['Title'] = movie_poster_data['Title']
# movie_poster_words['Poster'] = movie_poster_data['Poster']


In [56]:
movie_poster_detected_words = []
unsupported_image = 0
for image in image_list:
    word_list = []
    try:
        d = pytesseract.image_to_data(image, output_type=Output.DICT)['text']
        #print(d)
        grayed = get_grayscale(image)
        threshed = thresholding(grayed)
        openinged = opening(grayed)
        cannyed = canny(grayed)
        grayscale = pytesseract.image_to_data(grayed, output_type=Output.DICT)['text']
        threshscale = pytesseract.image_to_data(threshed, output_type=Output.DICT)['text']
        openingscale = pytesseract.image_to_data(openinged, output_type=Output.DICT)['text']
        cannyscale = pytesseract.image_to_data(cannyed, output_type=Output.DICT)['text']
        
        # word_list += d
        # word_list += grayscale
        # word_list += threshscale
        # word_list += openingscale
        # word_list += cannyscale


        # u can consider using this function instead since this will prevent duplicates word from being added from 2 images
      



        word_list.extend(x for x in d if x not in word_list)
        word_list.extend(x for x in grayscale if x not in word_list)
        word_list.extend(x for x in threshscale if x not in word_list)
        word_list.extend(x for x in openingscale if x not in word_list)
        word_list.extend(x for x in cannyscale if x not in word_list)
        
        # word_list = set(word_list)
        # word_list = list(set(word_list))
        print(word_list)
        movie_poster_detected_words.append(word_list)

    except Exception as e:
        print('Exception: ' + str(e))
        list_nothing = ['']
        movie_poster_detected_words.append(list_nothing)
        unsupported_image += 1



['', '  ', ' ', 'ai', 'ZS', '-', 'fi', 'as', '5', 'AAC', 'SOV', 'on', 'OLS', '*', 'ait', 'ERS', 'SVORY', 'at', 'ae', 'ere', 'iv', 'Sane', '>', 'Ss', '=', '~', 'oa', 'a', 'mae', 'ene']
['', 'Cee', 'et', 'ee', 'ert)', 'eae', 'oe', 'eT', 'ray', 'Re', 'ea', ' ', 'ya', '|', 'ba', 'Ea', 'Rhine', 'goed', 'raf', 'rrr', 'Beda', 'ried', 'A', '8', '®', 'Bt', '#', 'Sa', 'en', 'eg', 'CES', 'VIED', 'RAN', 'leet', 'UI', 'toy', 'cee', 'od', 'ed']
['', 'TACK', 'LDAMON', 'WALTER', 'ANT', 'TEM', 'ANNOMARGRET', 'SOPHIA', 'LOFIN', ' ', 'ry', 'pr', 'GRUMPIER', 'OLD', 'a', 'JACK', 'LMON', 'ANN', 'MARGRET', 'LOFEN', '|', 'gt', 'ih', 'gs', 'AN', 'HOP', 'OWN', 'WATER', 'NET', 'EA', 'WA', 'MAG', 'ELT', 'SOTIN', 'OPEN', 'r', 'ae', 'oe', 'MEN', 'AIBN', 'iar', 'BANG', 'SUNTAN', 'CC', 'TaN', 'DE', ',', 'Ma', ';', 'i;', 'eo', 'gis', 'he', 'in', 'OU', 'hy', ':)', 'i', 'am', '!', 'Sy', 's', 'A', 'Saat', 'eet', 'eV', 'es', 'GRUMPTER', 'ite', 'iat', 'ed', 'POR', 'Ui', 'hea', 'OS', 'ee', 'C', 'ANTES']
['', 'Whitney', 'Hou

KeyboardInterrupt: ignored

In [ ]:
movie_poster_words_temp = movie_poster_words.head(100)
movie_poster_words_trimmed = movie_poster_words_temp.copy()
movie_poster_words_trimmed['detected_words'] = movie_poster_detected_words

In [ ]:
movie_poster_words_trimmed.head(50)

In [ ]:
print('percentage of unsupported image ', float(unsupported_image)/100 )

In [ ]:
movie_poster_words_trimmed.to_csv('./pytesseract/movie_poster_detected_words.csv')

In [ ]:
       # ignore this part of the code first, I was testing something but it was not working
       
        # gray = cv2.cvtColor(cannyed, cv2.COLOR_BGR2GRAY) 
        # ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV) 
        # rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18)) 
        # dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1) 
        # contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,  
        #                                          cv2.CHAIN_APPROX_NONE) 
        # for cnt in contours: 
        #   x, y, w, h = cv2.boundingRect(cnt) 
            
        #   # Drawing a rectangle on copied image 
        #   rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2) 
            
        #   # Cropping the text block for giving input to OCR 
        #   cropped = im2[y:y + h, x:x + w] 
        #   text = pytesseract.image_to_string(cropped) 
        #   word_list.append(text)